In [1]:
import pandas as pd
import sys
from data_reader import fes_reader, perf_reader
from tqdm import tqdm
import bisect
import json
import datetime

In [2]:
# конвертация путей файлов в зависимости от системы
def replace_slash(file_path):
    platform = sys.platform
    slash_map = {'win32': '\\',
                'cygwin': '\\',
                'darwin': '/',
                'linux2': '/'}
    if platform not in slash_map.keys(): platform = 'linux2'
    return file_path.replace('\\', slash_map[platform])

In [3]:
perf_path = replace_slash('input_data\\' + 'armitz/miner/ARX_PERF.xlsx')
fes_path = replace_slash('input_data\\armitz/miner/QINF.xlsx')

In [4]:
perf_ints = perf_reader(perf_path)
fes_dict = fes_reader(fes_path)
SOIL_CUT = 60



started reading perf xl
done reading perf xl
started reading fes xl
done reading fes xl
